![header](https://i.imgur.com/I4ake6d.jpg)

# IN SITU BLACK SEA TRAINING

<div style="text-align: right"><i> 13-04-Part-four-out-of-five </i></div>

# BS `NRT` product/dataset: managing files (profilers)

***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction">Introduction</a></span></li>
        <li>
            <span><a href="#Setup" data-toc-modified-id="Setup">Setup</a></span>
            <ul>
                <li><span><a href="#Python-packages" data-toc-modified-id="Python-packages">Python packages</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Profilers-(PF)-data" data-toc-modified-id="Profilers-(PF)-data">Profilers (PF) data</a></span>
            <ul>
                <li><span><a href="#Reading-file" data-toc-modified-id="Reading-file">Reading file</a></span></li>
                <li><span><a href="#Data-visualization" data-toc-modified-id="Data-visualization">Data visualization</a></span>
                    <ul>
                        <li><span><a href="#Trajectory-animation" data-toc-modified-id="Trajectory-animation">Trajectory animation</a></span></li>
                        <li><span><a href="#1D-profile" data-toc-modified-id="1D-profile">1D profile</a></span></li>
                        <li><span><a href="#2D-profile" data-toc-modified-id="2D-profile">2D profile</a></span></li>
                        <li><span><a href="#3D-profile" data-toc-modified-id="3D-profile">3D profile</a></span></li>
                    </ul>
                </li>
           </ul>
        </li>
        <li><span><a href="#Wrap-up" data-toc-modified-id="Wrap-up">Wrap-up</a></span></li>
        <li><span><a href="#Feedback-survey" data-toc-modified-id="Feedback-survey">Feedback survey</a></span></li>
    </ul>
</div>

***

## Introduction

According to the [13-01-NearRealTtime-product-collections-overview.ipynb](13-01-NearRealTtime-product-collections-overview.ipynb) one of the data source types available are Profilers. Please use the notebook [13-02-NearRealTtime-product-subsetting-download](13-02-NearRealTtime-product-subsetting-download.ipynb) to download some files from Profilers ('PF' data type) and let's check its data. If you wanna skip the downloading part you can use the netCDF files available <a href="data" target="_blank">here</a> instead.  

## Setup

### Python packages

For the notebook to properly run we need to first load the next packages available from the Jupyter Notebook Ecosystem. Please run the `next cell`:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import IPython
import datetime
import pandas as pd
import os
import numpy as np
import xarray
import matplotlib.pyplot as plt
from matplotlib import colors
import folium
from folium import plugins
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If any of them raises any error it means you need to install the module first. For doing so please:
1. Open a new cell int he notebook
2. Run <i>`!conda install packageName --yes`</i> or <i>`!conda install -c conda-forge packageName --yes`</i> or <i>`!pip install packageName`</i>
3. Import again!
<br><br>
Example: <i>how-to-solve import error for json2html module </i>

![region.png](img/errorImporting.gif)

## Profilers (PF) data

Profilers are floating devices that are released into the ocean in other to measure a number of variables such as temperature and salinity. It drifts along the currents and sinks every now and then in the watter column to retrieve the variation of such parameters along the depth (profiles!).

<br>Here a self explanatory video from the Argo program:  `run the next cell`

In [ ]:
iframe = '<iframe width="100%" height="400" src="https://www.youtube.com/embed/WGbanFvBX38" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
IPython.display.HTML(iframe)

Let's see the data of one of the available profilers in the Black Sea.

### Reading the file

`Run the next cell` to see the profilers files already available in the /data folder:

In [ ]:
IPython.display.IFrame('data/files/PF', width='100%', height=350)

`Set one` of the above available `file name` and `run the next cells`:

In [ ]:
file = 'GL_PR_PF_3901852.nc'
path = os.path.join(os.getcwd(), 'data','files','PF', file)

In [ ]:
ds = xarray.open_dataset(path)
ds.close()
ds

The above one is an overview of the content of the file: variables, dimensions, coordinates, global attributes...
<br>Let's list now the available variables: `run the next cell`

In [ ]:
for var in ds.variables:
    print(var + ':' + ds[var].attrs['long_name'])

### Data visualization

Let's focus on one of the above variables to visualize its data!: `set one and run the next cell`

In [ ]:
param = 'TEMP_ADJUSTED'

In [ ]:
ds[param]

#### Trajectory animation

As stated before, the profilers are mobile platforms. <br>Let's check the overall trajectory by joining the sampling points (locations where a profile has been performed).

In In Situ TAC netCDFs all variables are linked to another called the same plus '_QC'. This 'twin' variable contains a quality flag for each value in the paired variable.`run the next cell` to check the flag values convention:

In [ ]:
pd.DataFrame(data=ds['POSITION_QC'].attrs['flag_values'],
             index=ds['POSITION_QC'].attrs['flag_meanings'].split(' '), columns=['quality flag'])

Users are recommended to use only the data flagged as 1, they so called 'good data'. Let's then check the available flags for the coordinates (time and position) to see if we need to get rid of not-good values: `run the next cells`

In [ ]:
ds['POSITION_QC'].plot(aspect=2, size=5)

In [ ]:
ds['TIME_QC'].plot(aspect=2, size=5)

From above, we see no flags values different from 1, so we are ready to go!. Be aware nevertheless, that in the event of other flags values, a selection of the data must be done. See next how:`run the next cell`

In [ ]:
lats = ds['LATITUDE'].where(ds['POSITION_QC'][0] == 1).values.tolist()
lons = ds['LONGITUDE'].where(ds['POSITION_QC'][1] == 1).values.tolist()
times = ds['TIME'].where(ds['TIME_QC'] == 1).values.tolist()
strtimes = ds['TIME'].where(ds['TIME_QC'] == 1).values[:]

Let's initialize now a geojson object that will represent our profiler: `run the next cell`

In [ ]:
profiler = {
    'type': 'Feature',
    'geometry': {
        'type': 'LineString',
        'coordinates': []#to be populated with lat,lon pairs
    },
    'properties': {
        'times': [] #to be populed with time strings
    }
}

Let's now popule it: `run the next cell`

In [ ]:
for time, strtime, lat, lon in zip(times, strtimes, lats, lons):
    base = [time,lat,lon]
    if(any(x is None for x in base)):
        continue
    if(any(np.isnan(x) for x in base)):
        continue
    profiler['properties']['times'].append(str(strtime)[:22])
    profiler['geometry']['coordinates'].append([lon, lat])

Let's now create a map to finally see the overall trajectory performed by the profiler:

In [ ]:
mean_lat, mean_lon = np.mean(lats), np.mean(lons)
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)
marker = plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': [profiler],
}, period='P1D', add_last_point=True, loop=False).add_to(m)
m

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you do not see any map when running the next cell please change your navigator (try chrome!).

#### 1D profile

Let's see the selected variable variation with depth in a specific time/position..

Set first cell the profile to check (`run the next cell`): 

In [ ]:
idx = 0 #First profile

Set a title for the plot (`run the next cell`):

In [ ]:
title = 'Profile from platform '+ds.platform_code+\
    ' \n time('+str(ds['TIME'].values[idx])[:19]+') '+\
    '& position('+str(ds['LATITUDE'].values[idx])+','+str(ds['LONGITUDE'].values[idx])+')'

Create plot (`run the next cell`):

In [ ]:
ds[param][idx].where(ds[param+'_QC'][idx]==1).plot.line(aspect=2, size=7, marker='o',y='DEPTH')
plt.title(title)
plt.gca().invert_yaxis()

#### 2D profile

Let's plot the selected variable variation with time and depth (no matter the position)!

Set a new title (`run the next cell`):

In [ ]:
title = '2D Profile from platform '+ds.platform_code+\
    ' \n times('+str(ds['TIME'].values[0])[:19]+' / '+str(ds['TIME'].values[-1])[:19]+') '

Create a 2D plot (`run the next cell`):

In [ ]:
ds[param].where(ds[param+'_QC'] == 1).plot(
    x='TIME', y='DEPTH', aspect=3, size=10, cmap='jet')
plt.title(title)
plt.gca().invert_yaxis()

#### 3D profile

Let's plot the selected variable variation with time, depth and position!

Set first a colormap (`run the next cell`):

In [ ]:
cmap = plt.cm.Spectral_r
norm = colors.Normalize(vmin=ds[param].where(ds[param+'_QC'] == 1).min(
).values.tolist(), vmax=ds[param].where(ds[param+'_QC'] == 1).max().values.tolist())

Specify the list of profiles to plot (`run the next cell`):

In [ ]:
profiles2plot = range(0, len(ds['TIME'])) #Defaults to all!
profiles2plot

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you face any problem with the next plot, try reducing the number of `profiles2plot`

Create the 3D plot (`run the next cell`):

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for a in profiles2plot:
    lat = ds['LATITUDE'].values.tolist()[a]
    lon = ds['LONGITUDE'].values.tolist()[a]
    temp = ds[param].where(ds[param+'_QC'] == 1).values.tolist()[a]
    depth = (-1*ds['DEPTH']).values.tolist()
    plt.scatter(lon*np.ones(len(depth)), lat*np.ones(len(depth)), zs=depth, zdir='z', s=20, c=temp, cmap=cmap, norm=norm)
cbar = plt.colorbar(orientation="horizontal", pad=0.02)
cbar.ax.set_xlabel(param)
ax.set_title(str(a+1)+' temperature profiles from platform ' + ds.id, y=1.08)
ax.set_zlabel('depth', labelpad=20, rotation=90)
ax.set_ylabel('latitude', labelpad=20)
ax.set_xlabel('longitude', labelpad=20)

***

## Wrap-up

So far you should already know how to deal with profile-like data from profilers platforms. <br> `If you don't please ask us! it is the moment!`

*** 

## Feedback survey

<div class="alert alert-block alert-success">
    <b>CONGRATULATIONS</b><br>

***
**IF IT'S 202025 PAST MIDDAY, PLEASE READ CAREFULLY BELOW LINES (ACTION FROM YOUR SIDE)**
***    
This training course is over but we'd love to hear from you about how we could improve it (topics, tools, storytelling, format, speed etc). 

We have prepared a little questionnaire to gather all your inputs, available here (just click on the hyperlink or execute the very last cell and click on `Answer`):
- https://tiny.cc/training-blk-insitu

We do thank you in advance for your kind collaboration :)

Greetings <3

In [ ]:
IPython.display.IFrame('https://tiny.cc/training-blk-insitu', width=900, height=500)